In [1]:
import sys
import os
from os.path import basename
from IPython.display import HTML
import matplotlib.pyplot as plt

import PIL
import import_ipynb
from Sampler import *
from Pix2Code import *


from Utility import *
from Compiler import *

importing Jupyter notebook from Sampler.ipynb
importing Jupyter notebook from Vocabulary.ipynb
importing Jupyter notebook from BeamSearch.ipynb
importing Jupyter notebook from Utils.ipynb
importing Jupyter notebook from Pix2Code.ipynb


Using TensorFlow backend.
c:\users\bhagya sree\anaconda3\envs\pix2code\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\bhagya sree\anaconda3\envs\pix2code\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\bhagya sree\anaconda3\envs\pix2code\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\bhagya sree

importing Jupyter notebook from Utility.ipynb
importing Jupyter notebook from Compiler.ipynb
importing Jupyter notebook from Node.ipynb


In [2]:
trained_weights_path = "WebModel"
trained_model_name = "pix2code"
eval_path = "eval_set"
output_path = "CodeWeb"
search_method = "greedy"
file_name = "Output"
meta_dataset = np.load("{}/meta_dataset.npy".format(trained_weights_path),allow_pickle=True)
input_shape = meta_dataset[0]
output_size = meta_dataset[1]
print(input_shape,output_size)

(256, 256, 3) 19


In [ ]:


model = pix2code(input_shape, output_size, trained_weights_path)
model.load(trained_model_name)

sampler = Sampler(trained_weights_path, input_shape, output_size, CONTEXT_LENGTH)


print("Model Loaded")

In [ ]:

FILL_WITH_RANDOM_TEXT = True
TEXT_PLACE_HOLDER = "[]"
def render_content_with_text(key, value):
        if FILL_WITH_RANDOM_TEXT:
            if key.find("btn") != -1:
                value = value.replace(TEXT_PLACE_HOLDER, Utility.get_random_text())
            elif key.find("title") != -1:
                value = value.replace(TEXT_PLACE_HOLDER, Utility.get_random_text(length_text=5, space_number=0))
            elif key.find("text") != -1:
                value = value.replace(TEXT_PLACE_HOLDER,
                                      Utility.get_random_text(length_text=56, space_number=7, with_upper_case=False))
        return value

In [ ]:
def on_button_clicked1(b):
    uploaded_filename = next(iter(uploader.value))
    content = uploader.value[uploaded_filename]['content']
    #print(uploader.value)
    
    input_path = eval_path+"/"+uploaded_filename
    img = PIL.Image.open(input_path)
    # Plot the image.
    plt.figure(figsize=(12,10))
    plt.imshow(img)
    plt.show()
    
    filename = eval_path+"/"+ basename(input_path)[:basename(input_path).find(".")]+".gui"
    #print(filename)
    
    print("Generating GUI....")
    f = open(filename, "r")
    result1 = f.read()
    #input_path = filename
    
    evaluation_img = Utils.get_preprocessed_img(input_path, IMAGE_SIZE)

    if search_method == "greedy":
        result, _ = sampler.predict_greedy(model, np.array([evaluation_img]))
        #print("Result greedy: {}".format(result))
    else:
        beam_width = int(search_method)
        print("Search with beam width: {}".format(beam_width))
        result, _ = sampler.predict_beam_search(model, np.array([evaluation_img]), beam_width=beam_width)
        print("Result beam: {}".format(result))
    
    if os.path.exists(filename):
        with open("{}/{}.gui".format(output_path, file_name), 'w') as out_f:
            out_f.write(result1)
    else:
        with open("{}/{}.gui".format(output_path, file_name), 'w') as out_f:
            out_f.write(result.replace(START_TOKEN, "").replace(END_TOKEN, ""))
    
    
    print("Converted To GUI")
    
    print("Generating HTML....")

    dsl_path = "assets/web-dsl-mapping.json"
    compiler = Compiler(dsl_path)


    input_file_path = "{}/{}.gui".format(output_path, file_name)
    output_file_path = "{}/{}.html".format(output_path, file_name)

    #print(input_file_path, output_file_path)
    compiler.compile(input_file_path, output_file_path, rendering_function=render_content_with_text)
    print("Converted To HTML")
    
def on_button_clicked(b):
    

    filename = 'upload.png'
    uploaded_filename = next(iter(uploader.value))
    content = uploader.value[uploaded_filename]['content']
    with open(filename, 'wb') as f: 
        f.write(content)
    
    img = PIL.Image.open(filename)
    # Plot the image.
    plt.figure(figsize=(12,10))
    plt.imshow(img)
    plt.show()
    
    print("Generating GUI....")
    input_path = filename
    print(filename)
    evaluation_img = Utils.get_preprocessed_img(input_path, IMAGE_SIZE)

    if search_method == "greedy":
        result, _ = sampler.predict_greedy(model, np.array([evaluation_img]))
        #print("Result greedy: {}".format(result))
    else:
        beam_width = int(search_method)
        print("Search with beam width: {}".format(beam_width))
        result, _ = sampler.predict_beam_search(model, np.array([evaluation_img]), beam_width=beam_width)
        print("Result beam: {}".format(result))


    with open("{}/{}.gui".format(output_path, file_name), 'w') as out_f:
        out_f.write(result.replace(START_TOKEN, "").replace(END_TOKEN, ""))
        
    print("Converted To GUI")
    
    print("Generating HTML....")

    dsl_path = "assets/web-dsl-mapping.json"
    compiler = Compiler(dsl_path)


    input_file_path = "{}/{}.gui".format(output_path, file_name)
    output_file_path = "{}/{}.html".format(output_path, file_name)

    #print(input_file_path, output_file_path)
    compiler.compile(input_file_path, output_file_path, rendering_function=render_content_with_text)
    print("Converted To HTML")

    #HTML(filename="CodeWeb/Output.html")

In [ ]:
import ipywidgets as widgets
from ipyupload import FileUpload
from IPython.display import display
from ipywidgets import Button, Layout

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    width='50%'
)
style = {'description_width': '150px'}

button = widgets.Button(description="Convert",button_style='danger',layout=Layout(width='30%', height='40px'))

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    border='solid',
                    width='75%')


title = widgets.HTML(
    value=" <h3>WEB PIX2CODE</h3>",
    placeholder='',
    description='',
)
label = widgets.HTML(
    value=" <b>Please Select Image</b>",
    placeholder='',
    description='',
)
    
uploader = FileUpload(


accept='',
multiple=False,
disabled=False, # default
# CSS transparently passed to button (in a fact a label element)
# e.g. 'color: darkblue; background-color: lightsalmon; width: 180px;'
style_button='color: white; background-color: green; width: 220px;' # default
)


#box = widgets.VBox([tab,button],layout=box_layout)
box = widgets.VBox([title,label,uploader,button],layout=box_layout)
display(box)
button.on_click(on_button_clicked1)



In [ ]:
HTML(filename="CodeWeb/Output.html")